## Longitudinal Education Outcomes (LEO) analysis on graduate earnings

### Analysis performed on 2021/22 data on industry of graduate employment, up to the 5-digit Standard Industry Classification (SIC) code level, available [here](https://content.explore-education-statistics.service.gov.uk/api/releases/20a83553-df0f-4fc1-e6d5-08dc0dccb030/files/018ba8f3-c614-4abb-aa1b-88244edd0c86) on GOV.UK

In [1]:
# import required libraries

import pandas as pd
import numpy as np

In [2]:
# load data into Pandas DataFrame

df = pd.read_excel("input/industry_tables_data_5_digit_SIC.xlsx")

In [3]:
# view first 5 rows of data

df.head()

,time_period,time_identifier,geographic_level,country_code,country_name,YAG,SECTIONNAME,group_name,group_number_3dig,sic_detailed,SIC_5dig,qualification_TR,sex,subject_name,ethnicity_major,prior_attainment,FSM,current_region,count,earnings_median
0,202122,Tax year,National,K02000001,United Kingdom,1 YAG,Accommodation and food service activities,Hotels and similar accommodation,551,Hotels and similar accommodation,55100,First degree,Female + male,"Agriculture, food and related studies",Total,Total,Total,Total,20,18600
1,202122,Tax year,National,K02000001,United Kingdom,1 YAG,Accommodation and food service activities,Holiday centres and villages,552,Holiday centres and villages,55201,First degree,Female + male,"Agriculture, food and related studies",Total,Total,Total,Total,0,c
2,202122,Tax year,National,K02000001,United Kingdom,1 YAG,Accommodation and food service activities,Other holiday and other collective accommodation,552,Other holiday and other collective accommodation,55209,First degree,Female + male,"Agriculture, food and related studies",Total,Total,Total,Total,5,c
3,202122,Tax year,National,K02000001,United Kingdom,1 YAG,Accommodation and food service activities,Other accommodation,559,Other accommodation,55900,First degree,Female + male,"Agriculture, food and related studies",Total,Total,Total,Total,0,c
4,202122,Tax year,National,K02000001,United Kingdom,1 YAG,Accommodation and food service activities,Licensed restaurants,561,Licensed restaurants,56101,First degree,Female + male,"Agriculture, food and related studies",Total,Total,Total,Total,20,18600


In [4]:
# explore columns

df.describe(include='all')

,time_period,time_identifier,geographic_level,country_code,country_name,YAG,SECTIONNAME,group_name,group_number_3dig,sic_detailed,SIC_5dig,qualification_TR,sex,subject_name,ethnicity_major,prior_attainment,FSM,current_region,count,earnings_median
count,726152.0,726152,726152,726152,726152,726152,726152,726152,726152.0,726152,726152,726152,726152,726152,726152,726152,726152,726152,726152.000000,726152
unique,NaN,1,1,1,1,4,22,703,263.0,703,703,4,3,38,7,11,4,15,NaN,917
top,NaN,Tax year,National,K02000001,United Kingdom,3 YAG,Wholesale and retail trade - repair of motor v...,Not known,477.0,Not known,NA,First degree,Female + male,Total,Total,Total,Total,Total,NaN,c
freq,NaN,726152,726152,726152,726152,186260,116982,5458,21909.0,5458,5458,635671,600799,76810,624851,573420,650240,539070,NaN,623335
mean,202122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.545752,NaN
std,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.574691,NaN
min,202122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
25%,202122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
50%,202122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
75%,202122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN


In [5]:
# drop columns that only have one value in them

df.drop(columns=['time_period', 'time_identifier', 'geographic_level', 'country_code', 'country_name', 'group_number_3dig', 'SIC_5dig'], inplace = True)

In [6]:
# explore YAG column
# YAG = Years after graduation

df['YAG'].value_counts()

YAG
3 YAG     186260
10 YAG    184761
5 YAG     184015
1 YAG     171116
Name: count, dtype: int64

In [7]:
# replace ' YAG' from each row and convert column to number

df["YAG"] = pd.to_numeric(df["YAG"].str.replace(" YAG", ""))

# rename the column for clarity in analysis

df.rename(columns={'YAG': 'years_after_graduation'}, inplace=True)

In [8]:
# explore SECTIONNAME column
# SECTIONNAME = Industry section (Standard Industry Classification, SIC section name)

df['SECTIONNAME'].value_counts()

SECTIONNAME
Wholesale and retail trade - repair of motor vehicles and motorcycles                                                         116982
Manufacturing                                                                                                                 112752
Professional, scientific and technical activities                                                                              73504
Information and communication                                                                                                  59371
Administrative and support service activities                                                                                  52523
Financial and insurance activities                                                                                             41165
Human health and social work activities                                                                                        39030
Education                                                

In [9]:
# remove rows where industry is not known

df.drop(df[df.SECTIONNAME == 'Not known'].index, inplace=True)

# rename the column to standardise format for analysis

df.rename(columns={'SECTIONNAME': 'section_name'}, inplace=True)

In [10]:
# explore group_name column
# group_name = Industry group name

df['group_name'].value_counts()

group_name
First-degree level higher education                                  5272
Hospital activities                                                  5226
Primary education                                                    5095
Management consultancy activities other than financial management    4947
General public administration activities                             4916
                                                                     ... 
Raising of camels and camelids                                          8
Manufacture of mortars                                                  8
Manufacture of knitted and crocheted fabrics                            8
Growing of tropical and subtropical fruits                              4
Printing of newspapers                                                  4
Name: count, Length: 702, dtype: int64

In [11]:
# explore sic_detailed column
# sic_detailed = Industry class/sub-class name

df['sic_detailed'].value_counts()

sic_detailed
First-degree level higher education                                  5272
Hospital activities                                                  5226
Primary education                                                    5095
Management consultancy activities other than financial management    4947
General public administration activities                             4916
                                                                     ... 
Raising of camels and camelids                                          8
Manufacture of mortars                                                  8
Manufacture of knitted and crocheted fabrics                            8
Growing of tropical and subtropical fruits                              4
Printing of newspapers                                                  4
Name: count, Length: 702, dtype: int64

In [12]:
# explore qualification_TR column
# qualification_TR = Qualification level

df['qualification_TR'].value_counts()

qualification_TR
First degree          631229
Level 7 (taught)       66728
Level 8                15729
Level 7 (research)      7008
Name: count, dtype: int64

In [13]:
# Level 6 = degree or equivalent
# Level 7 = masters or equivalent
# Level 8 = doctorate or equivalent

# this analysis will focus on level 6 qualifications only as it is the most common qualification choice for higher education
# link: https://explore-education-statistics.service.gov.uk/find-statistics/progression-to-higher-education-or-training

df.drop(df[df.qualification_TR != 'First degree'].index, inplace=True)

# drop the qualification_TR column now there's only one value in it

df.drop(columns=['qualification_TR'], inplace = True)

In [14]:
# explore sex column
# sex = Graduate sex

df['sex'].value_counts()

sex
Female + male    559232
Male              36292
Female            35705
Name: count, dtype: int64

In [15]:
# explore subject_name column
# subject_name = Subject studied

df['subject_name'].value_counts()

subject_name
Total                                         65227
Business and management                       41724
Creative arts and design                      32916
Engineering                                   29682
Computing                                     24436
Performing arts                               23291
Media, journalism and communications          23096
English studies                               22247
Sociology, social policy and anthropology     22234
History and archaeology                       22029
Psychology                                    21275
Geography, earth and environmental studies    20018
Law                                           19894
Biosciences                                   19791
Sport and exercise sciences                   19453
Economics                                     17499
Politics                                      17100
Languages and area studies                    16839
Mathematical sciences                         16371

In [16]:
# explore ethnicity_major column
# ethnicity_major = Ethnicity broad

df['ethnicity_major'].value_counts()

ethnicity_major
Total                                          530684
White                                           43145
Asian / Asian British                           17843
Black / African / Caribbean / Black British     13214
Mixed / Multiple ethnic groups                  12146
Unknown                                          8158
Other ethnic group                               6039
Name: count, dtype: int64

In [17]:
# explore prior_attainment column
# prior_attainment = Prior attainment level

df['prior_attainment'].value_counts()

prior_attainment
Total                    479710
300-359 points            24315
240-299 points            24284
180-239 points            17985
BTEC                      17979
360 points                13767
Not known                 13607
1 or 2 A level passes     13094
Other                     12813
4 As or more               7588
Below 180 points           6087
Name: count, dtype: int64

In [18]:
# explore FSM column
# FSM = Free school meal eligibility status

df['FSM'].value_counts()

FSM
Total        555725
non-FSM       39376
Not known     19470
FSM           16658
Name: count, dtype: int64

In [19]:
# explore current_region column
# current_region = Current region

df['current_region'].value_counts()

current_region
Total                              445781
London                              24594
South East                          23134
North West                          21034
East of England                     18824
Yorkshire and The Humber            18163
West Midlands                       17937
South West                          17350
East Midlands                       16543
North East                          10610
All regions in Wales                 7351
All regions in Scotland              4872
All regions in Northern Ireland      3861
Outside UK                           1002
Not Known                             173
Name: count, dtype: int64

In [20]:
# explore count column
# count = Number of graduates

df['count'].value_counts()

count
0        385101
5        127412
10        39866
15        18307
20        11569
          ...  
6795          1
12760         1
3665          1
14340         1
2325          1
Name: count, Length: 631, dtype: int64

In [21]:
# rename the column for clarity in analysis

df.rename(columns={'count': 'number_of_graduates'}, inplace=True)

In [22]:
# explore earnings_median column
# earnings_median = Median earnings

df['earnings_median'].value_counts()

earnings_median
c         542067
24800       1584
26600       1513
26300       1479
24100       1471
           ...  
51200          1
69800          1
64500          1
46100          1
170100         1
Name: count, Length: 803, dtype: int64

In [23]:
# replace 'c' values with NumPy not a number (NaN)
# c = data has been supressed due to small numbers

df["earnings_median"] = df["earnings_median"].replace("c", np.NaN)

In [24]:
# explore combinations of demographic variables in the data

df.groupby(['sex', 'ethnicity_major', 'prior_attainment', 'FSM', 'current_region']).size().reset_index().rename(columns={0:'count'})

,sex,ethnicity_major,prior_attainment,FSM,current_region,count
0,Female,Total,Total,Total,Total,35705
1,Female + male,Asian / Asian British,Total,Total,Total,17843
2,Female + male,Black / African / Caribbean / Black British,Total,Total,Total,13214
3,Female + male,Mixed / Multiple ethnic groups,Total,Total,Total,12146
4,Female + male,Other ethnic group,Total,Total,Total,6039
5,Female + male,Total,1 or 2 A level passes,Total,Total,13094
6,Female + male,Total,180-239 points,Total,Total,17985
7,Female + male,Total,240-299 points,Total,Total,24284
8,Female + male,Total,300-359 points,Total,Total,24315
9,Female + male,Total,360 points,Total,Total,13767


In [25]:
# it isn't possible in this dataset to explore combinations of demographic variables
# this analysis will focus on differences between male and female graduates to further explore the well-understood gap between male and female earnings

df.drop(df[df.sex == 'Female + male'].index, inplace=True)

In [26]:
# remove the remaining demographic colummns that are no longer required

df.drop(columns=['ethnicity_major', 'prior_attainment', 'FSM', 'current_region'], inplace = True)

In [27]:
# explore the combinations of sex and subject_name

df.groupby(['sex', 'subject_name']).size().reset_index().rename(columns={0:'count'})

,sex,subject_name,count
0,Female,"Agriculture, food and related studies",922
1,Female,Allied health,1023
2,Female,"Architecture, building and planning",720
3,Female,Biosciences,1211
4,Female,Business and management,2036
...,...,...,...
67,Male,Psychology,936
68,Male,"Sociology, social policy and anthropology",1080
69,Male,Sport and exercise sciences,1392
70,Male,Total,2610


In [28]:
# create dataframe with subjects included except the totals

df_sex_subjects = df.drop(df[df.subject_name == 'Total'].index)

In [29]:
# check data after transformations

df_sex_subjects.head()

,years_after_graduation,section_name,group_name,sic_detailed,sex,subject_name,number_of_graduates,earnings_median
46360,1,Accommodation and food service activities,Hotels and similar accommodation,Hotels and similar accommodation,Female,"Agriculture, food and related studies",15,19000.0
46361,1,Accommodation and food service activities,Hotels and similar accommodation,Hotels and similar accommodation,Male,"Agriculture, food and related studies",5,NaN
46362,1,Accommodation and food service activities,Holiday centres and villages,Holiday centres and villages,Female,"Agriculture, food and related studies",0,NaN
46363,1,Accommodation and food service activities,Other holiday and other collective accommodation,Other holiday and other collective accommodation,Female,"Agriculture, food and related studies",0,NaN
46364,1,Accommodation and food service activities,Other holiday and other collective accommodation,Other holiday and other collective accommodation,Male,"Agriculture, food and related studies",0,NaN


In [30]:
# create dataframe with only the totals across subjects

df_sex = df.drop(df[df.subject_name != 'Total'].index)

In [34]:
# remove the subject_name column that is no longer required

df_sex.drop(columns=['subject_name'], inplace=True)

In [35]:
# check data after transformations

df_sex.head()

,years_after_graduation,section_name,group_name,sic_detailed,sex,number_of_graduates,earnings_median
113517,1,Accommodation and food service activities,Hotels and similar accommodation,Hotels and similar accommodation,Female,695,17900.0
113518,1,Accommodation and food service activities,Hotels and similar accommodation,Hotels and similar accommodation,Male,415,18200.0
113519,1,Accommodation and food service activities,Holiday centres and villages,Holiday centres and villages,Female,55,16800.0
113520,1,Accommodation and food service activities,Holiday centres and villages,Holiday centres and villages,Male,30,17200.0
113521,1,Accommodation and food service activities,Youth hostels,Youth hostels,Female,5,NaN


In [36]:
# output the data to a CSV

df_sex.to_csv("output/graduate_outcomes_industry_sex_salary.csv", index = False)
df_sex_subjects.to_csv("output/graduate_outcomes_industry_sex_salary_subject.csv", index = False)